In [1]:
import numpy as np
import pandas as pd
import requests
import json
import xlrd


In [62]:
# Extracting data from the business registry in Norway  - not sufficiently working as it max extracts 10.000 samples
baseUrl = "https://data.brreg.no/enhetsregisteret/api/enheter"
fromRegistryDateQuery = "fraRegistreringsdatoEnhetsregisteret=2000-01-01"
#toRegistryDateQuery = "tilRegistreringsdatoEnhetsregisteret=2007-01-01"
#fromFoundedDateQuery = "fraStiftelsesdato=2000-01-01"
sizeQuery = "size=10000"
sortQuery = "sort=fraStiftelsesdato"
#orgQuery = "organisasjonsform=ADOS,ANNA,ANS,AS,ASA,ENK"


url = baseUrl + '?' + fromFoundedDateQuery + '&' + sizeQuery
response = requests.get(url)
print(response)

<Response [200]>


In [ ]:
data = response.json()
len(data)

In [ ]:
# Number of businesses
units = data['_embedded']
print(units)

In [ ]:
# Downloading a xlsx with all business - takes too long
url = "https://data.brreg.no/enhetsregisteret/api/enheter/lastned/regneark"
headers = {'Accept': 'application/vnd.brreg.enhetsregisteret.enhet+vnd.openxmlformats-officedocument.spreadsheetml.sheet;charset=UTF-8'}
response = requests.get(url)
print(response)

In [ ]:
workbook = xlrd.open_workbook(file_contents=response.content)  # open workbook
worksheet = workbook.sheet_by_index(0)  # get first sheet
first_row = worksheet.row(0)  # you can iterate over rows of a worksheet as well


In [68]:
# Downloading a g-zip with all business - works well!
url = "https://data.brreg.no/enhetsregisteret/api/enheter/lastned"
headers = {'Accept': 'application/vnd.brreg.enhetsregisteret.enhet.v1+gzip;charset=UTF-8'}
response = requests.get(url)
print(response)

<Response [200]>


In [69]:
print(response.headers)

{'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Content-Disposition': 'attachment; filename=enheter_alle.json.gz', 'Accept-Ranges': 'bytes', 'Content-Type': 'application/gzip;charset=UTF-8', 'Content-Length': '85238578', 'Date': 'Sat, 13 Jun 2020 20:11:36 GMT', 'Set-Cookie': '427fb3af220e1b20a96f703941cf5ddb=b79f79947c88b6586dc7122c5fe9434c; path=/; HttpOnly; Secure, BRjEE1=!mVKiud78JUsCK8nLIICWUnHw+aorreskyscWbiAO+R2437KA4rekG6W9r91cFj9/VUjnfs8/LDuakw==; path=/; Httponly; Secure', 'Cache-control': 'private', 'Strict-Transport-Security': 'max-age=157680000; includeSubDomains'}


In [72]:
open('enheter_alle.json.gz', 'wb').write(response.content)

85238578

In [ ]:
# Extracting 10.000 of all updates by date - not working as all updates are "unknown until mid 2019"
url = 'https://data.brreg.no/enhetsregisteret/api/oppdateringer/enheter?dato=2019-05-01T00:00:00.000Z&size=10000'
response = requests.get(url)
print(response)

In [ ]:
# Number of businesses
data = response.json()
units = data['_embedded']['oppdaterteEnheter']
len(units)

In [ ]:
#print(json.dumps(parsed['_embedded']['enheter'][0], indent=4, sort_keys=True))

In [66]:
print(json.dumps(data, indent=4, sort_keys=True))

{
    "_embedded": {
        "enheter": [
            {
                "_links": {
                    "self": {
                        "href": "https://data.brreg.no/enhetsregisteret/api/enheter/920162673"
                    }
                },
                "antallAnsatte": 0,
                "forretningsadresse": {
                    "adresse": [
                        "c/o Tuva Ystad Geder\u00f8",
                        "Steinst\u00f8lveien 17"
                    ],
                    "kommune": "STAVANGER",
                    "kommunenummer": "1103",
                    "land": "Norge",
                    "landkode": "NO",
                    "postnummer": "4027",
                    "poststed": "STAVANGER"
                },
                "institusjonellSektorkode": {
                    "beskrivelse": "Ideelle organisasjoner",
                    "kode": "7000"
                },
                "konkurs": false,
                "maalform": "Nynorsk",
            

In [54]:
with open('data1.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [61]:
df_batch = pd.read_json('data1.json')
df_batch.to_csv ('from_2017-12-21_to_2006-01-16.csv', index = None)